# Fine-Tuning LLaMA-2 with QLoRA and PEFT

Este notebook demonstra como realizar o fine-tuning do modelo LLaMA-2 utilizando as técnicas QLoRA e PEFT. O objetivo é treinar o modelo para gerar respostas baseadas em instruções específicas.

## 1. Instalação de Dependências
Instalamos as bibliotecas necessárias para o treinamento.

In [ ]:
!pip install -U trl git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes einops wandb transformers accelerate

## 2. Configuração do Ambiente
Definimos variáveis de ambiente e verificamos a GPU disponível.

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Configuração do token de autenticação
os.environ["HUGGING_FACE_HUB_TOKEN"] = "<SEU_TOKEN_AQUI>"
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

# Verificar GPU
!nvidia-smi

## 3. Carregamento do Dataset
Utilizamos o dataset `Ekkologico/phyton-clean-tiny-codes` para o treinamento.

In [ ]:
from datasets import load_dataset

dataset_name = 'Ekkologico/phyton-clean-tiny-codes'
dataset = load_dataset(dataset_name, split="train", use_auth_token=True)

## 4. Configuração do Modelo
Carregamos o modelo LLaMA-2 com quantização em 4 bits para otimizar o uso de memória.

In [ ]:
model_name = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto"
)
model.config.use_cache = False

## 5. Configuração do Tokenizer
Carregamos o tokenizer correspondente ao modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

## 6. Configuração do PEFT
Definimos os parâmetros para o fine-tuning utilizando LoRA.

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

## 7. Configuração do Treinamento
Definimos os argumentos para o treinamento, como batch size, taxa de aprendizado e número de passos.

In [ ]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=100,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    eval_accumulation_steps=1,
    gradient_checkpointing=True
)

## 8. Treinamento do Modelo
Utilizamos o `SFTTrainer` para realizar o fine-tuning.

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="combined",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train()

## 9. Salvando o Modelo
Salvamos o modelo treinado para reutilização futura.

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("outputs")

## 10. Testando o Modelo
Geramos uma resposta com o modelo treinado.

In [ ]:
text = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Create a Python function that takes in a string and a list of words and returns true if the string contains all the words in the list. ### Input: 'This is a test', ['test', 'this', 'is']"
inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))